In [ ]:
!pip install arxiv transformers sentencepiece torch pandas tqdm
import arxiv
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch
from datetime import datetime, timedelta

def get_papers_summary():
    search = arxiv.Search(
        query="cat:cs.CL",
        max_results=5000,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = []
    for result in tqdm(search.results(), total=5000, desc="Fetching papers"):
        papers.append({
            'title': result.title,
            'abstract': result.summary,
            'published': result.published,
            'authors': [a.name for a in result.authors],
            'pdf_url': result.pdf_url
        })
        if len(papers) >= 5000:
            break
    return papers

device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=device
)

def extract_technical_details(text):
    prompt = f"""
    Extract the following from this research paper abstract:
    1. PROBLEM: Identify the core technical challenge (1-2 sentences)
    2. SOLUTION: Describe the proposed architecture/algorithm (2-3 sentences)

    Text: {text[:1500]}  # Truncate to avoid OOM

    Format exactly as:
    PROBLEM: [description]
    SOLUTION: [description including architecture and algorithms]
    """
    result = summarizer(prompt, max_length=300, min_length=100, do_sample=False)
    return result[0]['summary_text']

def create_enhanced_dataset(papers):
    data = []

    for paper in tqdm(papers, desc="Processing papers"):
        try:
            extracted = extract_technical_details(paper['abstract'])

            problem = extracted.split('PROBLEM:')[-1].split('SOLUTION:')[0].strip()
            solution = extracted.split('SOLUTION:')[-1].strip()

            data.append({
                'title': paper['title'],
                'authors': ', '.join(paper['authors']),
                'date': paper['published'].strftime('%Y-%m-%d'),
                'abstract': paper['abstract'],
                'problem': problem,
                'solution': solution,
                'url': paper['pdf_url']
            })
        except Exception as e:
            print(f"Error processing {paper['title']}: {str(e)}")
            continue

    return pd.DataFrame(data)

if __name__ == "__main__":
    papers = get_papers_summary()
    df = create_enhanced_dataset(papers)

    csv_path = "NLP_Problem_Solution_arxiv.csv"
    df.to_csv(csv_path, index=False)
    print(f"\nSaved {len(df)} papers to {csv_path}")
    print("Sample record:")
    print(df.iloc[0])

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 s

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-1-843d45f4032c>:20: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in tqdm(search.results(), total=5000, desc="Fetching papers"):
Processing papers:  70%|██████▉   | 3492/5000 [1:33:12<42:36,  1.70s/it]